In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math

def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

# predict 0

In [36]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
#train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)


# 강한 추세성보이는 칼럼 중 일부 제거
'''
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)
'''

# {'onehot', 'onehot-dense', 'ordinal'}
from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="kmeans", random_state=1414)

featurest = train_T_31_x.columns[1:]
enc.fit(train_T_31_x[featurest])
binned_df_train = enc.transform(train_T_31_x[featurest])
binned_df_test = enc.transform(test_T_31_x[featurest])
for i, featurest in enumerate(featurest):
    train_T_31_x[f"{featurest}_binned"] = binned_df_train[:,i]
    test_T_31_x[f"{featurest}_binned"] = binned_df_test[:,i]

enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="kmeans", random_state=1414)
features = trainA_31_x.columns[1:]
enc.fit(trainA_31_x[features])
binned_df_train = enc.transform(trainA_31_x[features])
binned_df_test = enc.transform(testA_31_x[features])
for i, feature in enumerate(features):
    trainA_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    testA_31_x[f"{feature}_binned"] = binned_df_test[:,i]

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 4233) (67, 4233) (343, 1355) (239, 1355)


In [37]:
# binary
trainA_31_w.loc[(trainA_31_w['Y_Class']==2),'Y_Class']=1
train_T_31_w.loc[(train_T_31_w['Y_Class']==2),'Y_Class']=1
traint = pd.concat([train_T_31_x,train_T_31_w],axis = 1)
traina = pd.concat([trainA_31_x,trainA_31_w],axis = 1)

In [38]:
corr_df = traint.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_t = []#성장[성장[0]<0.002].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_t = []
for col in traint.columns[1:678] :
    col_corr = traint[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :        
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_t.append(col+'_binned')
        else :
            droppc_t.append(col)
dropt = set(list(under_05per_t)+droppc_t+['LINE'])
print(len(under_05per_t), len(droppc_t))

0 379


In [39]:
corr_df = traina.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_a = []#성장[성장[0]<0.007].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_a = []
for col in traina.columns[1:2117] :
    col_corr = traina[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :    
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_a.append(col+'_binned')
        else :
            droppc_a.append(col)
dropa = set(list(under_05per_a)+droppc_a+['LINE'])
print(len(under_05per_a), len(droppc_a))

0 1964


In [40]:
trainA_31_x = trainA_31_x.drop(columns=dropa)
testA_31_x = testA_31_x.drop(columns=dropa)
train_T_31_x = train_T_31_x.drop(columns=dropt)
test_T_31_x = test_T_31_x.drop(columns=dropt)

In [41]:
print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 2268) (67, 2268) (343, 975) (239, 975)


In [71]:
catreg1 = CatBoostRegressor(random_state=1000,
                           depth=5,
                           verbose=500,
                           iterations=3000,
                           learning_rate=0.022)

catreg2 = CatBoostRegressor(random_state=3337,
                           depth=4,
                           verbose=500,
                           iterations=4000,
                           learning_rate=0.021)

In [72]:
catreg1.fit(trainA_31_x, trainA_31_y)
pred_a = catreg1.predict(testA_31_x)

0:	learn: 0.0098925	total: 11.6ms	remaining: 34.8s
500:	learn: 0.0020167	total: 4.37s	remaining: 21.8s
1000:	learn: 0.0005304	total: 8.72s	remaining: 17.4s
1500:	learn: 0.0001582	total: 13s	remaining: 13s
2000:	learn: 0.0000508	total: 17.3s	remaining: 8.66s
2500:	learn: 0.0000163	total: 21.7s	remaining: 4.32s
2999:	learn: 0.0000052	total: 26s	remaining: 0us


In [73]:
catreg2.fit(train_T_31_x, train_T_31_y)
pred_t = catreg2.predict(test_T_31_x)

0:	learn: 0.0047276	total: 3.47ms	remaining: 13.9s
500:	learn: 0.0018275	total: 1.45s	remaining: 10.1s
1000:	learn: 0.0007818	total: 2.89s	remaining: 8.65s
1500:	learn: 0.0003844	total: 4.31s	remaining: 7.18s
2000:	learn: 0.0001885	total: 5.75s	remaining: 5.74s
2500:	learn: 0.0000966	total: 7.2s	remaining: 4.31s
3000:	learn: 0.0000502	total: 8.65s	remaining: 2.88s
3500:	learn: 0.0000264	total: 10.1s	remaining: 1.44s
3999:	learn: 0.0000140	total: 11.5s	remaining: 0us


In [74]:
for i in range(0,len(pred_a)):
    if pred_a[i] <= 0.525066667  :
        pred_a[i] = 0
    else : pred_a[i] = 1

In [75]:
for i in range(0,len(pred_t)):
    if pred_t[i] <= 0.525066667  :
        pred_t[i] = 0
    else : pred_t[i] = 1

In [76]:
print(len(pred_a) - sum(pred_a),(len(pred_a) - sum(pred_a))/len(pred_a))

30.0 0.44776119402985076


In [77]:
print(len(pred_t) - sum(pred_t),(len(pred_t) - sum(pred_t))/len(pred_t))

10.0 0.04184100418410042


# predict 2

In [79]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
# train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

# {'onehot', 'onehot-dense', 'ordinal'}
from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="kmeans", random_state=1414)

featurest = train_T_31_x.columns[1:]
enc.fit(train_T_31_x[featurest])
binned_df_train = enc.transform(train_T_31_x[featurest])
binned_df_test = enc.transform(test_T_31_x[featurest])
for i, featurest in enumerate(featurest):
    train_T_31_x[f"{featurest}_binned"] = binned_df_train[:,i]
    test_T_31_x[f"{featurest}_binned"] = binned_df_test[:,i]

enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="kmeans", random_state=1414)
features = trainA_31_x.columns[1:]
enc.fit(trainA_31_x[features])
binned_df_train = enc.transform(trainA_31_x[features])
binned_df_test = enc.transform(testA_31_x[features])
for i, feature in enumerate(features):
    trainA_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    testA_31_x[f"{feature}_binned"] = binned_df_test[:,i]

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 4233) (67, 4233) (343, 1355) (239, 1355)


In [80]:
# binary
trainA_31_w.loc[(trainA_31_w['Y_Class']==0),'Y_Class']=1
train_T_31_w.loc[(train_T_31_w['Y_Class']==0),'Y_Class']=1
train_O_31_w.loc[(train_O_31_w['Y_Class']==0),'Y_Class']=1
traint = pd.concat([train_T_31_x,train_T_31_w],axis = 1)
traina = pd.concat([trainA_31_x,trainA_31_w],axis = 1)

In [81]:
corr_df = traint.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_t = []#성장[성장[0]<0.001].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_t = []
for col in traint.columns[1:678] :
    col_corr = traint[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :        
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_t.append(col+'_binned')
        else :
            droppc_t.append(col)
dropt = set(list(under_05per_t)+droppc_t+['LINE'])
print(len(under_05per_t), len(droppc_t))

0 379


In [82]:
corr_df = traina.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_a = []#성장[성장[0]<0.001].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_a = []
for col in traina.columns[1:2117] :
    col_corr = traina[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :    
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_a.append(col+'_binned')
        else :
            droppc_a.append(col)
dropa = set(list(under_05per_a)+droppc_a+['LINE'])
print(len(under_05per_a), len(droppc_a))

0 1964


In [83]:
trainA_31_x = trainA_31_x.drop(columns=dropa)
testA_31_x = testA_31_x.drop(columns=dropa)
train_T_31_x = train_T_31_x.drop(columns=dropt)
test_T_31_x = test_T_31_x.drop(columns=dropt)

In [84]:
print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 2268) (67, 2268) (343, 975) (239, 975)


In [100]:
catreg3 = CatBoostRegressor(random_state=1234,
                           depth=5,
                           #eval_metric='MAE',
                           verbose=500,
                           iterations=5000,
                           learning_rate=0.008)

catreg4 = CatBoostRegressor(random_state=7770,
                           depth=6,
                           #eval_metric='MAE',
                           verbose=500,
                           iterations=5000,
                           learning_rate=0.003)

In [86]:
catreg3.fit(trainA_31_x, trainA_31_y)
pred_a2 = catreg3.predict(testA_31_x)

0:	learn: 0.0099205	total: 10.6ms	remaining: 53s
500:	learn: 0.0055271	total: 3.91s	remaining: 35.1s
1000:	learn: 0.0033408	total: 7.73s	remaining: 30.9s
1500:	learn: 0.0020223	total: 11.6s	remaining: 27s
2000:	learn: 0.0012592	total: 15.4s	remaining: 23.1s
2500:	learn: 0.0007824	total: 19.3s	remaining: 19.2s
3000:	learn: 0.0005028	total: 23.1s	remaining: 15.4s
3500:	learn: 0.0003188	total: 26.9s	remaining: 11.5s
4000:	learn: 0.0002085	total: 30.8s	remaining: 7.69s
4500:	learn: 0.0001370	total: 34.6s	remaining: 3.84s
4999:	learn: 0.0000903	total: 38.5s	remaining: 0us


In [101]:
catreg4.fit(train_T_31_x, train_T_31_y)
pred_t2 = catreg4.predict(test_T_31_x)

0:	learn: 0.0047437	total: 11.3ms	remaining: 56.5s
500:	learn: 0.0038130	total: 2.92s	remaining: 26.2s
1000:	learn: 0.0032562	total: 5.9s	remaining: 23.6s
1500:	learn: 0.0028107	total: 8.86s	remaining: 20.7s
2000:	learn: 0.0023947	total: 11.8s	remaining: 17.8s
2500:	learn: 0.0019792	total: 14.9s	remaining: 14.9s
3000:	learn: 0.0016029	total: 18s	remaining: 12s
3500:	learn: 0.0013080	total: 21.1s	remaining: 9.03s
4000:	learn: 0.0010529	total: 24.2s	remaining: 6.05s
4500:	learn: 0.0008336	total: 27.4s	remaining: 3.04s
4999:	learn: 0.0006692	total: 30.6s	remaining: 0us


In [88]:
for i in range(0,len(pred_a2)):
    if pred_a2[i] < 0.534950794   :     
        pred_a2[i] = 1
    else : pred_a2[i] = 2

In [89]:
-len(pred_a2) + sum(pred_a2)

4.0

In [102]:
for i in range(0,len(pred_t2)):
    if pred_t2[i] < 0.534950794   :     
        pred_t2[i] = 1
    else : pred_t2[i] = 2

In [103]:
-len(pred_t2) + sum(pred_t2)

9.0

In [106]:
testA_31['Y_2'] = pred_a2
test_T_31['Y_2'] = pred_t2

testA_31['Y_0'] = pred_a
test_T_31['Y_0'] = pred_t

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_0']==0  ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_2']==2  ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_0']==0  ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_2']==2  ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('k5.csv',index=False)

In [107]:
ddd = pd.read_csv('k5.csv')

In [115]:
kd = pd.read_csv('corrr.csv')

In [254]:
ddd['Y_Class'].value_counts()/len(ddd)

1    0.829032
0    0.132258
2    0.038710
Name: Y_Class, dtype: float64

In [108]:
ddd['Y_Class'].value_counts()

1    257
0     40
2     13
Name: Y_Class, dtype: int64

In [116]:
kd[kd['Y_Class']==0]

,PRODUCT_ID,Y_Class
7,TEST_007,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
15,TEST_015,0
35,TEST_035,0
37,TEST_037,0
39,TEST_039,0
41,TEST_041,0
42,TEST_042,0


In [110]:
ddd[ddd['Y_Class']==0]

,PRODUCT_ID,Y_Class
3,TEST_003,0
7,TEST_007,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
15,TEST_015,0
22,TEST_022,0
28,TEST_028,0
32,TEST_032,0
35,TEST_035,0


In [111]:
ddd[ddd['Y_Class']==2]

,PRODUCT_ID,Y_Class
6,TEST_006,2
11,TEST_011,2
38,TEST_038,2
78,TEST_078,2
95,TEST_095,2
119,TEST_119,2
126,TEST_126,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2


In [117]:
kd[kd['Y_Class']==2]

,PRODUCT_ID,Y_Class
11,TEST_011,2
12,TEST_012,2
38,TEST_038,2
78,TEST_078,2
95,TEST_095,2
134,TEST_134,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
183,TEST_183,2
